In [58]:
import requests
import json
import pandas as pd
from fpdf import FPDF
import re

In [71]:
import warnings
warnings.filterwarnings('ignore')

def extract():
    api_key = eval(open('config.txt').read())['auth']
    player_stats = requests.get(f'https://api.sportsdata.io/v3/nba/stats/json/PlayerSeasonStatsByTeam/2023/LAL?key={api_key}').json()
    standings = requests.get(f'https://api.sportsdata.io/v3/nba/scores/json/Standings/2023?key={api_key}').json()
    return player_stats, standings


def write_to_json_file(data, standings):
    with open('nba_stats.json', 'w') as f:
        json.dump(data, f)
    with open('standings.json', 'w') as f:
        json.dump(standings, f)


def transform(data, standings):
    # Transforming player stats
    df_players = pd.DataFrame(data)
    interesting_columns = ['Position', 'Games', 'FantasyPoints', 'Minutes', 'TwoPointersMade', 'TwoPointersAttempted', 'TwoPointersPercentage', 'ThreePointersMade', 'ThreePointersAttempted', 'ThreePointersPercentage', 'FreeThrowsMade', 'FreeThrowsAttempted', 'FreeThrowsPercentage', 'OffensiveRebounds', 'DefensiveRebounds', 'Rebounds', 'Assists', 'Steals', 'BlockedShots', 'Turnovers', 'PersonalFouls', 'Points', 'PlayerEfficiencyRating']
    df_players = df_players.set_index('Name')
    df_players = df_players[interesting_columns]
    column_names = {
        'Position': 'Pos',
        'Games': 'G',
        'FantasyPoints': 'FP',
        'Minutes': 'Min',
        'TwoPointersMade': '2PM',
        'TwoPointersAttempted': '2PA',
        'TwoPointersPercentage': '2P%',
        'ThreePointersMade': '3PM',
        'ThreePointersAttempted': '3PA',
        'ThreePointersPercentage': '3P%',
        'FreeThrowsMade': 'FTM',
        'FreeThrowsAttempted': 'FTA',
        'FreeThrowsPercentage': 'FT%',
        'OffensiveRebounds': 'OffReb',
        'DefensiveRebounds': 'DefReb',
        'Rebounds': 'Reb',
        'Assists': 'Ast',
        'Steals': 'Stl',
        'BlockedShots': 'Blk',
        'Turnovers': 'TO',
        'PersonalFouls': 'PF',
        'Points': 'Pts',
        'PlayerEfficiencyRating': 'Rtg'
    }
    df_players.index.name = None
    for _ in range(len(interesting_columns)):
        df_players.rename(columns=column_names, inplace=True)
    # Change column names
    df_players.sort_values(by=['FP'], ascending=False, inplace=True)

    # Transforming standings
    df_rnk = pd.DataFrame(standings)
    df_rnk['Team'] = df_rnk['City'] + ' ' + df_rnk['Name']
    df_rnk_conf = df_rnk[df_rnk['Conference'] == df_rnk[df_rnk['Team'] == 'Los Angeles Lakers']['Conference'].values[0]]
    for column in ['ConferenceWins', 'ConferenceLosses', 'HomeWins', 'HomeLosses', 'AwayWins', 'AwayLosses', 'LastTenWins', 'LastTenLosses', 'Streak']:
        df_rnk_conf[column] = df_rnk_conf[column].astype(str)
    df_rnk_conf['Conf'] = df_rnk_conf["ConferenceWins"] + '-' + df_rnk_conf["ConferenceLosses"]
    df_rnk_conf['Home'] = df_rnk_conf['HomeWins'] + '-' + df_rnk_conf['HomeLosses']
    df_rnk_conf['Away'] = df_rnk_conf['AwayWins'] + '-' + df_rnk_conf['AwayLosses']
    df_rnk_conf['L10'] = df_rnk_conf['LastTenWins'] + '-' + df_rnk_conf['LastTenLosses']
    interesting_columns = ['Team', 'Wins', 'Losses', 'Percentage', 'GamesBack', 'Conf', 'Home', 'Away', 'L10', 'Streak']
    column_names = {
        'Team': 'Team',
        'Wins': 'W',
        'Losses': 'L',
        'Percentage': 'Pct',
        'GamesBack': 'GB',
        'Conf': 'Conf',
        'Home': 'Home',
        'Away': 'Away',
        'L10': 'L10',
        'Streak': 'Strk'
    }
    df_rnk_conf = df_rnk_conf[interesting_columns]
    for _ in range(len(interesting_columns)):
        df_rnk_conf.rename(columns=column_names, inplace=True)
        df_rnk_conf.sort_values(by=['Pct'], ascending=False, inplace=True)
    df_rnk_conf = df_rnk_conf.set_index('Team')
    df_rnk_conf.index.name = None
    df_rnk_conf['Strk'] = 'W' + df_rnk_conf['Strk']
    df_rnk_conf['Strk'] = df_rnk_conf['Strk'].str.replace('W-', 'L')
    return df_players, df_rnk_conf


def create_pdf(df):
    pdf = FPDF()
    pdf.add_page()
    pdf.set_font('Arial', 'B', 16)
    pdf.cell(40, 10, 'Los Angeles Lakers')
    pdf.ln()
    pdf.ln()
    pdf.set_font('Arial', 'B', 12)
    pdf.cell(40, 10, 'Season 2022')
    pdf.ln()
    pdf.set_font('Arial', 'B', 10)
    pdf.cell(40, 10, 'Player stats')
    pdf.ln()
    lengths = {
        'Pos': 6,
        'G': 4,
        'FP': 8,
        'Min': 7,
        '2PM': 7,
        '2PA': 7,
        '2P%': 6,
        '3PM': 7,
        '3PA': 7,
        '3P%': 6,
        'FTM': 7,
        'FTA': 7,
        'FT%': 6,
        'OffReb': 9,
        'DefReb': 9,
        'Reb': 7,
        'Ast': 7,
        'Stl': 6,
        'Blk': 6,
        'TO': 7,
        'PF': 7,
        'Pts': 7,
        'Rtg': 6
    }
    pdf.set_font('Arial', 'B', 5)
    pdf.cell(18, 8, 'Player', border=1)
    for key, value in lengths.items():
        pdf.cell(value, 8, key, border=1)
    pdf.ln()
    pdf.set_font('Arial', '', 5)
    for index, row in df.iterrows():
        pdf.cell(18, 4, index, border=1)
        for key, value in lengths.items():
            pdf.cell(value, 4, str(row[key]), border=1)
        pdf.ln()
    pdf.multi_cell(0, 2, '\n* Pos = Position, G = Games, FP = Fantasy Points, Min = Minutes, 2PM = Two Pointers Made, 2PA = Two Pointers Attempted, 2P% = Two Pointers Percentage, 3PM = Three Pointers Made, 3PA = Three Pointers Attempted, 3P% = Three Pointers Percentage, FTM = Free Throws Made, FTA = Free Throws Attempted, FT% = Free Throws Percentage, OffReb = Offensive Rebounds, DefReb = Defensive Rebounds, Reb = Rebounds, Ast = Assists, Stl = Steals, Blk = Blocks, TO = Turnovers, PF = Personal Fouls, Pts = Points, Rtg = Player Efficiency Rating')
    pdf.output('nba_stats.pdf', 'F')
    


In [38]:
stats, standings = extract()


In [39]:
write_to_json_file(stats, standings)


In [72]:
df, df2 = transform(stats, standings)
display(df)
display(df2)


,Pos,G,FP,Min,2PM,2PA,2P%,3PM,3PA,3P%,...,OffReb,DefReb,Reb,Ast,Stl,Blk,TO,PF,Pts,Rtg
Anthony Davis,C,16,1353.3,913,257.8,432.4,99.1,11.1,31.6,35.1,...,91.5,247.8,339.3,66.5,43.2,58.2,56.5,78.2,700.1,58.2
LeBron James,SF,12,921.1,707,156.3,282.7,92.0,41.6,144.7,47.7,...,29.9,148.0,177.9,128.1,24.9,15.0,78.2,41.6,513.9,42.3
Russell Westbrook,PG,17,889.4,798,111.4,256.1,72.3,36.6,119.7,50.9,...,29.9,118.1,148.0,209.5,29.9,11.6,103.1,71.5,417.4,30.1
Lonnie Walker IV,SG,16,647.2,853,116.4,211.2,91.6,53.2,143.0,61.9,...,13.8,51.6,59.9,49.9,26.6,11.6,31.6,46.6,455.7,29.2
Austin Reaves,SG,18,528.5,846,64.9,99.8,108.1,31.6,83.2,63.2,...,21.6,76.5,98.1,63.2,13.3,16.6,33.3,46.6,302.7,25.0
Troy Brown Jr.,SF,16,417.1,692,56.5,101.4,92.6,28.3,88.1,53.4,...,21.6,101.4,123.1,26.6,20.0,0.0,26.6,48.2,216.2,18.2
Patrick Beverley,PG,14,342.6,637,11.6,36.6,52.9,16.6,69.8,39.6,...,15.0,76.5,91.5,66.5,20.0,16.6,23.3,71.5,96.5,11.0
Wenyen Gabriel,PF,16,313.3,373,51.6,71.5,119.9,8.3,16.6,49.9,...,21.6,64.9,86.5,20.0,15.0,13.3,20.0,51.6,143.0,30.3
Kendrick Nunn,PG,16,226.0,373,36.6,79.8,76.2,24.9,84.8,48.9,...,2.8,34.9,36.6,34.9,5.5,2.8,38.2,21.6,158.0,11.4
Dennis Schröder,PG,5,134.0,195,16.6,39.9,69.3,11.1,18.3,60.5,...,2.8,11.6,13.3,26.6,8.3,0.0,11.1,11.6,74.8,24.3


,W,L,Pct,GB,Conf,Home,Away,L10,Strk
Phoenix Suns,13,6,0.684,0.0,11-3,11-1,2-5,6-4,W4
Denver Nuggets,12,7,0.632,1.0,10-4,4-2,8-5,6-4,W2
Memphis Grizzlies,12,8,0.600,1.5,7-5,7-2,5-6,5-5,W2
New Orleans Pelicans,11,8,0.579,2.0,7-5,6-3,5-5,6-4,L1
Los Angeles Clippers,12,9,0.571,2.0,9-8,7-5,5-4,6-4,W1
Sacramento Kings,10,8,0.556,2.5,4-5,6-3,4-5,7-3,L2
Portland Trail Blazers,11,9,0.550,2.5,8-4,4-4,7-5,4-6,L1
Utah Jazz,12,10,0.545,2.5,11-6,6-2,6-8,3-7,L4
Golden State Warriors,11,10,0.524,3.0,8-6,9-1,2-9,7-3,W3
Minnesota Timberwolves,10,10,0.500,3.5,5-7,5-6,5-4,5-5,L2


In [ ]:
create_pdf(df)